1. 인삼 모델을 만든다.

2. 모델의 

Library

In [17]:
import os
import json
import shutil

import cv2
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import random

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import StratifiedKFold
from torch.nn.modules.module import Module

import warnings

warnings.filterwarnings("ignore")

Check

In [6]:
dfdf = pd.read_csv('train_under.csv')

df_test_상 = dfdf[dfdf['label'] == '상'].sample(n=1000, random_state=42)
df_test_중 = dfdf[dfdf['label'] == '중'].sample(n=1000, random_state=42)
df_test_하 = dfdf[dfdf['label'] == '하'].sample(n=1000, random_state=42)
df_test_최하 = dfdf[dfdf['label'] == '최하'].sample(n=1000, random_state=42)

df_test = pd.concat([df_test_상, df_test_중, df_test_하, df_test_최하])

df_train = dfdf.drop(df_test.index)

df_test.to_csv('df_test.csv', index=False)
df_train.to_csv('df_train.csv', index=False)

print("df_test와 df_train 파일이 성공적으로 생성되었습니다.")

df_test와 df_train 파일이 성공적으로 생성되었습니다.


In [5]:
dfdf['label'].value_counts()

상     4263
중     4263
최하    4263
하     4263
Name: label, dtype: int64

File tranforming..

In [2]:
train_dir = 'D:/249.진안홍삼 품질 데이터/01-1.정식개방데이터/Validation/01.원천데이터/'
label_dir = 'D:/249.진안홍삼 품질 데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/'

train_folder = os.listdir(train_dir)
label_folder = os.listdir(label_dir)


dest_dir = 'C:/workspace/Insam/test'

test_df = pd.DataFrame(columns=['file_path', 'label'])

for folder in tqdm(train_folder):
    folder_path = os.path.join(train_dir, folder)
    label = folder.split('_')[1]

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        try:
            with Image.open(file_path) as img:
                if img.mode == 'RGB':
                    new_file_name = f"{label}_{file_name}"
                    dest_file_path = os.path.join(dest_dir, new_file_name)
                    shutil.copy(file_path, dest_file_path)
                    test_df = test_df.append({'file_path': dest_file_path, 'label': label}, ignore_index=True)
                    
        except IOError:
            print(f"Error opening file {file_path}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 404/404 [03:48<00:00,  1.77it/s]


Model?

In [18]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        self.label_mapping = {'상': 0, '중': 1, '하': 2, '최하': 3}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df['file_path'].iloc[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        target_label = self.df['label'].iloc[idx]
        target = self.label_mapping[target_label]
        target = torch.tensor(target, dtype=torch.long)
        return image, target
    
mytransform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(0, 360)),
    transforms.RandomPerspective(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

myvaltransform =transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CNN_Model(nn.Module):
    def __init__(self, class_n, rate=0.2):
        super(CNN_Model, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b7')
        self.dropout = nn.Dropout(rate)
        self.output_layer = nn.Linear(in_features=1000, 
                                      out_features=class_n, bias=True)

    def forward(self, inputs):
        output = self.output_layer(self.dropout(self.model(inputs)))
        return output
    
def train_step(model, batch_item, epoch, training, class_weight=None):
    img, label = batch_item
#     img = torch.tensor(img, dtype=torch.float32).detach()
#     label = torch.tensor(self.label, dtype=torch.long)
    img = img.to(device)
    label = label.to(device)
    if training:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step()
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label)
    
    # 정확도 계산
    pred = torch.argmax(output, dim=1)
    correct = (pred == label).float() 
    accuracy = correct.mean() 

    return loss, accuracy
    
def predict(models,dataset):
    for fold,model in enumerate(models):
        model.eval()
    tqdm_dataset = tqdm(enumerate(dataset))
    training = False
    results = []
    for batch, batch_item in tqdm_dataset:
        img = batch_item['img'].to(device)
        for fold,model in enumerate(models):
            with torch.no_grad():
                if fold ==0:
                    output = model(img)
                else:
                    output = output+model(img)
        output = 0.2*output
        output = torch.tensor(torch.argmax(output, axis=-1), dtype=torch.int32).cpu().numpy()
        results.extend(output)
    return results

train

In [11]:
if __name__ == '__main__':
    device = torch.device("cuda")
    batch_size = 16
    random_seed=42
    learning_rate = 1e-4
    epochs = 30
    
    model = CNN_Model(4).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    torch.manual_seed(random_seed)

    train_dataset = CustomDataset(csv_file='./df_train.csv', transform=mytransform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    test_dataset = CustomDataset(csv_file='./df_test.csv', transform=mytransform)
    test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    print("## ready to dataset to train## ")
    
    training = True
    loss_plot, val_loss_plot = [], []
    acc_plot, val_acc_plot = [], []

    # Training
    best_val_loss = np.inf 
    for epoch in range(epochs):
        total_loss, total_acc = 0, 0
        total_batches = 0

        for batch_item in tqdm(train_loader):
            batch_loss, batch_acc = train_step(model, batch_item, epoch, training)
            total_loss += batch_loss.item() 
            total_acc += batch_acc.item() 
            total_batches += 1 

        epoch_loss = total_loss / total_batches
        epoch_acc = total_acc / total_batches

        loss_plot.append(epoch_loss)
        acc_plot.append(epoch_acc)

        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")
        
    model.eval() 
    total_val_loss = 0
    total_val_batches = 0
    with torch.no_grad():
        for batch_item in tqdm(test_loader):
            batch_loss, _ = train_step(model, batch_item, epoch, False)  # training=False
            total_val_loss += batch_loss.item()
            total_val_batches += 1

    val_loss = total_val_loss / total_val_batches

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print(f"Epoch {epoch+1}: New best validation loss: {val_loss:.4f}. Saving model...")
        torch.save(model.state_dict(), "best_model.pth")

    print(f"Epoch {epoch+1}/{epochs} - Val Loss: {val_loss:.4f}")


Loaded pretrained weights for efficientnet-b7
## ready to dataset to train## 


100%|████████████████████████████████████████████████████████████| 816/816 [03:39<00:00,  3.72it/s]


Epoch 1/30 - Loss: 0.8937, Accuracy: 0.6069


100%|████████████████████████████████████████████████████████████| 816/816 [03:34<00:00,  3.80it/s]


Epoch 2/30 - Loss: 0.6829, Accuracy: 0.7145


...

Epoch 29/30 - Loss: 0.1996, Accuracy: 0.9236


100%|████████████████████████████████████████████████████████████| 816/816 [03:36<00:00,  3.77it/s]


Epoch 30/30 - Loss: 0.1837, Accuracy: 0.9323


100%|████████████████████████████████████████████████████████████| 816/816 [03:02<00:00,  4.47it/s]


Epoch 30: New best validation loss: 0.1624. Saving model...
Epoch 30/30 - Val Loss: 0.1624


In [12]:
dfdf

,Unnamed: 0,file_path,label
0,4058,C:/workspace/Insam/train\상_20220831_Xray_00035...,상
1,2046,C:/workspace/Insam/train\상_20220804_Xray_00231...,상
2,3177,C:/workspace/Insam/train\상_20220816_Xray_00091...,상
3,4534,C:/workspace/Insam/train\상_20220915_Xray_00138...,상
4,2935,C:/workspace/Insam/train\상_20220811_Xray_00301...,상
...,...,...,...
17047,135209,C:/workspace/Insam/train\하_20220901_Xray_00282...,하
17048,128532,C:/workspace/Insam/train\하_20220817_Xray_00102...,하
17049,136378,C:/workspace/Insam/train\하_20220906_Xray_00052...,하
17050,129960,C:/workspace/Insam/train\하_20220819_Xray_00116...,하


Predict

In [13]:
class CNN_Model(nn.Module):
    def __init__(self, class_n, rate=0.2):
        super(CNN_Model, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b7')
        self.dropout = nn.Dropout(rate)
        self.output_layer = nn.Linear(in_features=1000, out_features=class_n, bias=True)

    def forward(self, inputs):
        output = self.output_layer(self.dropout(self.model(inputs)))
        return output

mytransform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model = CNN_Model(4).to('cuda')
model.load_state_dict(torch.load('./best_model.pth'))
model.eval()

image_path = './test/상_20220706_Xray_002467.jpg'
image = Image.open(image_path)
image = mytransform(image).unsqueeze(0).to('cuda')

with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output, 1)
    label_mapping = {0: '상', 1: '중', 2: '하', 3: '최하'}
    predicted_label = label_mapping[predicted.item()]

print(f"The predicted label for '{image_path}' is {predicted_label}")

Loaded pretrained weights for efficientnet-b7
The predicted label for './test/상_20220706_Xray_002467.jpg' is 상


In [14]:
from tqdm import tqdm

test_dir = 'C://workspace//Insam//test/' 
test_files = os.listdir(test_dir)

test_df = pd.DataFrame(columns=['file_path', 'label'])

for file_name in tqdm(test_files):
    file_path = os.path.join(test_dir, file_name)
    label = file_name.split('_')[0]
    test_df = test_df.append({'file_path': file_path, 'label': label}, ignore_index=True)
    
test_df.to_csv('test_df.csv', index=False)

print("test.csv 파일이 성공적으로 생성되었습니다.")

100%|██████████████████████████████████████████████████████| 17539/17539 [00:07<00:00, 2312.57it/s]

test.csv 파일이 성공적으로 생성되었습니다.


In [15]:
test_df

,file_path,label
0,C://workspace//Insam//test/상_20220706_Xray_002...,상
1,C://workspace//Insam//test/상_20220706_Xray_002...,상
2,C://workspace//Insam//test/상_20220706_Xray_002...,상
3,C://workspace//Insam//test/상_20220708_Xray_000...,상
4,C://workspace//Insam//test/상_20220708_Xray_001...,상
...,...,...
17534,C://workspace//Insam//test/하_20220930_Xray_003...,하
17535,C://workspace//Insam//test/하_20220930_Xray_003...,하
17536,C://workspace//Insam//test/하_20220930_Xray_003...,하
17537,C://workspace//Insam//test/하_20220930_Xray_003...,하


In [16]:
model = CNN_Model(4).to('cuda')
model.load_state_dict(torch.load('./best_model.pth'))
model.eval()

test_df = pd.read_csv('test_df.csv')
correct_predictions = 0

for index, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_path = row['file_path']
    true_label = row['label']

    # 이미지 불러오기 및 전처리
    image = Image.open(image_path)
    image = mytransform(image).unsqueeze(0).to('cuda')

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        label_mapping = {0: '상', 1: '중', 2: '하', 3: '최하'}
        predicted_label = label_mapping[predicted.item()]

    if predicted_label == true_label:
        correct_predictions += 1

accuracy = correct_predictions / len(test_df)
print(f"Accuracy: {accuracy:.4f}")

Loaded pretrained weights for efficientnet-b7


 12%|██████▉                                                  | 2142/17539 [01:09<08:06, 31.65it/s]

In [19]:
class CNN_Model(nn.Module):
    def __init__(self, class_n, rate=0.2):
        super(CNN_Model, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b7')
        self.dropout = nn.Dropout(rate)
        self.output_layer = nn.Linear(in_features=1000, out_features=class_n, bias=True)

    def forward(self, inputs):
        output = self.output_layer(self.dropout(self.model(inputs)))
        return output

model = CNN_Model(4).to('cuda')
model.load_state_dict(torch.load('./best_model.pth'))
model.eval()

test_df = pd.read_csv('test_df.csv')
test_sample_df = test_df.sample(n=1000, random_state=42)
correct_predictions = 0

for index, row in tqdm(test_sample_df.iterrows(), total=len(test_sample_df)):
    image_path = row['file_path']
    true_label = row['label']

    image = Image.open(image_path)
    image = mytransform(image).unsqueeze(0).to('cuda')

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        label_mapping = {0: '상', 1: '중', 2: '하', 3: '최하'}
        predicted_label = label_mapping[predicted.item()]

    if predicted_label == true_label:
        correct_predictions += 1

accuracy = correct_predictions / len(test_sample_df)
print(f"Accuracy: {accuracy:.4f}")

Loaded pretrained weights for efficientnet-b7


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:31<00:00, 31.41it/s]

Accuracy: 0.8590
